In [73]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [74]:
#list of excel files downloaded from Tweet Archivist
filenames = ["all tweet data_thru July.xlsx","VR_export_200707_200727.xlsx"]
month = 7
#Hashtag(s) in lower case
excludedHTs = ["protectourvote","hr1","voteforourlives"]

In [75]:
def import_files(filenames):
    '''(list of strings -> df) Takes list of filenames, imports each one into pandas,
    then concatenates them into a single df
    '''
    
    #import dfs and put in a list
    df_list = []
    for filename in filenames:
        filedf = pd.read_excel(filename)
        df_list.append(filedf)
    
    #concat list if contains more than 1 df
    if len(df_list) > 1:
        df = pd.concat(df_list)
    else:
        df = df_list[0]
            
    return df


def month_filter(df, month):
    '''(df, integer > df) Takes the Tweet Archvisit df and a month, and filters the df to only include 
    Tweets in that month'''

    #Convert date column from string to datetime object
    df["Local Time Stamp"] = pd.to_datetime(df["Local Time Stamp"])

    #Extract month and year from the datetime column and map to new columns
    df["Month"] = df["Local Time Stamp"].map(lambda x: x.month)
    df["Year"] = df["Local Time Stamp"].map(lambda x: x.year)

    #filter for only the current month
    df = df[df['Month'] == month]

    return df


def get_hashtags(df, excludedHT):
    '''(df, string > df) Takes Tweet Archivist df, splits hashtag column, and returns a new df with each hashtag as a row, 
    excluding the hashtag being analyzed'''

    #Split hashtag column, drop nas and put in list
    split_htlist = df.Hashtags.str.split(" ").dropna().to_list()

    #Flatten list of lists
    htlist = [item for sublist in split_htlist for item in sublist]

    #Remove Protectourvote
    hashtags = pd.DataFrame(list(filter(lambda a: a != excludedHT, htlist)))
    
    return hashtags


def get_words(df):
    '''(df > df) Takes Tweet Archivist df, splits Tweet text column, removes hashtags, usernames
    non-alpha strings and stopwords'''
    
    #Make tweet text lower case
    df.Text = df.Text.str.lower()

    #Split text column, drop nas and put in list
    split_wordlist = df.Text.str.split(" ").dropna().to_list()

    #Flatten list of lists
    wordlist = [item for sublist in split_wordlist for item in sublist]

    #htlistwithhash = ["#" + word for word in htlist]
    
    #remove hashtags
    hashlist = [word for word in wordlist if word.startswith("#")]
    wordlist =  [word for word in wordlist if word not in hashlist]
    #remove usernames
    userlist = [word for word in wordlist if word.startswith("@")]
    wordlist = [word for word in wordlist if word not in userlist]

    #make wordlist into single string
    words = ",".join(wordlist)

    #Use NLTK tokenize to split string
    words = word_tokenize(words) 

    #Remove non-alpha words
    words = [word for word in words if word.isalpha()]

    #Set stopwords and remove
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    #create new df of all words
    words = pd.DataFrame(list(filter(lambda a: a != "rt", words)), columns = ["Words"])

              
    #Remove stray unneeded words
    words = words[words.Words != 'amp']
    words = words[words.Words != 'https']
              
    return words


In [76]:
#import and concatenate file(s)
df = import_files(filenames)

#filter for current month
df = month_filter(df, month)

#Use regex to create extract hashtags to Hashtags column
#Tweet Archivists' Hashtags column seems to leave some out
df["Hashtags"] = df["Text"].str.findall(r'#(\w+)')
#join the list of strings returned by findall, and make lower case
df['Hashtags'] = df["Hashtags"].apply(lambda x: ' '.join([str(i) for i in x])).str.lower()


#For hashtags in the list, filter for only tweets with that hashtag, 
#then get hashtags, words & tweet counts
for excludedHT in excludedHTs:

    #filter for only DFs with that particular hashtag
    htdf = df[df["Hashtags"].fillna("").str.contains(excludedHT)]
    
    #get tuple of hashtags df and hashtags list
    hashtags = get_hashtags(htdf, excludedHT)

    #get df of words
    words = get_words(htdf)
    
    #export df, word and hashtag lists to excel
    htdf.to_excel(excludedHT+" data.xlsx", sheet_name="#protecto",index=False)
    words.to_excel(excludedHT+" Tweet words.xlsx", index=False)
    hashtags.to_excel(excludedHT+" Hashtags.xlsx", index=False)

    print(excludedHT)
    print("Total tweets")
    print(len(htdf))
    print("Unique users")
    print(len(set(htdf["User Name"])))
    print("Potential Impressions")
    print(sum(htdf["Follower Count"]))
    print("\n")

/Users/lauranixon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


protectourvote
Total tweets
4161
Unique users
3078
Potential Impressions
34830976


hr1
Total tweets
621
Unique users
544
Potential Impressions
2672173




/Users/lauranixon/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://t.co/tga0nt9uvzyoung%20badgers%20are%20needed%20to%20be%20poll%20workers,%20as%20the%20%22older%20folks%22%20who%20typically%20do%20it,%20aren't%20able%20to,%20due%20to%20being%20high%20risk%20for%20covid-19.time%20is%20of%20the%20essence!%20%20apply%20now!%20%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/lauranixon/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://t.co/mjdqyxqpur%20please%20watch%20this%20on%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/lauranixon/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://t.co/klobhafjkigotta%20love%20gop.%20people%20are%20being%20kicked%20out%20of%20their%20homes%20an

voteforourlives
Total tweets
35377
Unique users
24627
Potential Impressions
124185778


